In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
!pip install "tensorflow>=1.15,<2.0"

In [0]:
import os
os.chdir('drive/My Drive/research/skin/')


In [0]:
pwd

'/content/drive/My Drive/research/skin'

In [0]:
import numpy as np
import pickle

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
import os.path
import fnmatch
import itertools
import functools
from keras.models import Model

Using TensorFlow backend.


In [0]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess =  tf.compat.v1.Session(config=config)

In [0]:
EPOCHS = 300
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((224, 224))
image_size = 0
width=224
height=224
depth=3

In [0]:
def load_image_files(container_path, dimension=(224, 224)):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    label_list = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            if str(file).split(".")[1] != 'jpg':
                continue
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
#             print(img_resized.shape)
            flat_data.append(img_resized) 
            images.append(img_resized)
            target.append(i)
            label_list.append(str(direc).split("/")[1])
#             print(label_list)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 label=label_list,
                     DESCR=descr)

In [0]:
image_dataset = load_image_files("Dataset/")

In [0]:
image_size = len(image_dataset.images)
print(image_size)

992


In [0]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(image_dataset.label)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [0]:
np_image_list = np.array(image_dataset.images, dtype=np.float16) / 225.0

In [0]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.30, random_state = 42)

[INFO] Spliting data to train, test


In [0]:
top_layers_checkpoint_path = 'cp.top.best.hdf5'
fine_tuned_checkpoint_path = 'cp.fine_tuned.best.hdf5'
new_extended_inception_weights = 'final_weights.hdf5'

In [0]:
aug = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [0]:
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
if os.path.exists(top_layers_checkpoint_path):
    model.load_weights(top_layers_checkpoint_path)
    print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

Checkpoint 'cp.top.best.hdf5' loaded.


In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint
mc_top = ModelCheckpoint(top_layers_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


In [0]:
model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[mc_top]
    )


In [0]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [0]:
mc_fit = ModelCheckpoint(fine_tuned_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

if os.path.exists(fine_tuned_checkpoint_path):
    model.load_weights(fine_tuned_checkpoint_path)
    print ("Checkpoint '" + fine_tuned_checkpoint_path + "' loaded.")

In [0]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [0]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[mc_fit]
    )


In [0]:
model.save_weights(new_extended_inception_weights)

In [0]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [0]:
print("Statistics of train Data: ")
print("Total image uses to train: ", len(x_train))
for i in range(len(label_binarizer.classes_)):
    print(label_binarizer.classes_[i]," : ", len([ x_train[j] for j in range(len(x_train)) if y_train[j][i] == 1]))
    

In [0]:
print("Statistics of test Data: ")
print("Total image uses to test: ", len(x_test))
for i in range(len(label_binarizer.classes_)):
    print(label_binarizer.classes_[i]," : ", len([ x_test[j] for j in range(len(x_test)) if y_test[j][i] == 1]))
    